In [ ]:
!pip install pandas 
!pip install seaborn

In [ ]:
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as  pd
import io
from PIL import Image, ImageFilter
import pandas as pd
import numpy as np
#from google.colab import files


# Inputting data from .CSV

In [ ]:
#Taking input of cloud excel file 

df = pd.read_csv('clouds_train_small.csv')
print(df)

#Pulling images from AWS bucket 

In [ ]:
def loadWeatherFile(filename, image_width = -1):
  import urllib.request
  import io
  from PIL import Image
  url = "http://uindy-weathercam.s3.us-east-2.amazonaws.com/" + filename
  fh = urllib.request.urlopen(url)
  if image_width == -1:
    return Image.open(io.BytesIO(fh.read()))
  else:
    im = Image.open(io.BytesIO(fh.read()))
    width, height = im.size
    image_height = image_width / width * height
    im = im.resize( (int(image_width), int(image_height)), Image.ANTIALIAS)
    return im

#Loading the file .jpg name then the labels 
df = df[['Cloud File', 'Label']]
df = df.astype({'Label': 'int32'}) # 
#if the label is 3 (rain picture) or 5 (bad picture) they're not accepted 
df = df.loc[(df.Label != 5) & (df.Label != 3)].reset_index(drop=True)
df.loc[df.Label == 4, 'Label'] = 3

df = df.sample(frac=1, random_state=2).reset_index(drop=True)

print(df)
print(df.dtypes)

In [ ]:
from multiprocessing import Pool, TimeoutError 

#load all ofthe files from the bucket, calling the above function
def f(filename):
    arr = np.array(loadWeatherFile(filename,200))  # <<---------- Image width! Can play with this to resize images for less CPU/GPU power 
    #making all the pixels from 0-1
    arr = arr / 255.0
    return arr

image_filenames = df[:]['Cloud File'].tolist()
with Pool(processes=12) as pool:      
    image_arr = pool.map(f, image_filenames)
print("Done processing images")
   
#making the np array of the 2D images 
image_features = np.asarray(image_arr)
print(image_features.shape)

#loading the labels so they're with the correct image 
image_labels = df[:(image_features.shape[0])]['Label']
print(image_labels.shape)

#showing the first image in the array 
plt.imshow(image_features[0])
plt.show()

# Creation of Model 

In [ ]:
#Creation of the model

model = None 

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=4, kernel_size=5, activation='relu',
                               padding='same'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=3),
    tf.keras.layers.Conv2D(filters=4, kernel_size=5,
                               activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=3),
    tf.keras.layers.Conv2D(filters=4, kernel_size=5,
                               activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(928, activation='relu' ),
    tf.keras.layers.Dense(256, activation='relu' ),
    tf.keras.layers.Dense(128, activation='relu' ),
    tf.keras.layers.Dense(units=4),#, kernel_initializer=init)
])

# Compilation of Model

In [ ]:
# Attemp to predict something - this will allow the model to know the input shape for the summary later!
model.predict(image_features) 

model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'],
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)) 
print(model.summary())
history = None


# Training

In [ ]:
# Train
history = model.fit(
    image_features,
    image_labels,
    epochs=20,
    # Suppress logging.
    verbose=1,
    # Calculate validation results on 20% of the training data.
    validation_split = 0.1,
    )

# Results Graph

In [ ]:
print(history.history.keys())

def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
  plt.ylim([0, 2])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)
  plt.show()

plot_loss(history)